# Advanced Topics

In this notebook, we will explore some advanced topics in `pybind11`, a lightweight header-only library that exposes C++ types in Python and vice versa. The topics covered include:

1. **Inheritance**: How to expose C++ class hierarchies to Python.
2. **Polymorphism**: Understanding how `pybind11` handles dynamic dispatch.
3. **Exception Handling**: Translating C++ exceptions to Python exceptions and vice versa.
4. **Multithreading**: Using `pybind11` in multi-threaded C++ applications.

For each topic, we will provide code examples and explanations to ensure a comprehensive understanding.

## Inheritance

Let's consider a simple example where we have a base class `Animal` and a derived class `Dog`. We want to expose both of these classes to Python using `pybind11`.

```cpp
#include <pybind11/pybind11.h>

class Animal {
public:
    Animal(const std::string &name) : name(name) {}
    virtual std::string sound() const { return "Unknown"; }
protected:
    std::string name;
};

class Dog : public Animal {
public:
    Dog(const std::string &name) : Animal(name) {}
    std::string sound() const override { return "Woof!"; }
};

PYBIND11_MODULE(example, m) {
    pybind11::class_<Animal>(m, "Animal")
        .def(pybind11::init<const std::string &>())
        .def("sound", &Animal::sound);

    pybind11::class_<Dog, Animal>(m, "Dog")
        .def(pybind11::init<const std::string &>())
        .def("sound", &Dog::sound);
}
```

In the code above, we first define the base class `Animal` and the derived class `Dog`. The `PYBIND11_MODULE` macro is used to create a Python module. Within this module, we use the `pybind11::class_` template to bind the `Animal` and `Dog` classes. Notice how the derived class `Dog` is specified to inherit from `Animal` in the `pybind11::class_` template.

## Polymorphism

Consider the previous `Animal` and `Dog` classes. The `sound` method in the `Animal` class is a virtual function, which is overridden in the `Dog` class. This allows for polymorphic behavior.

When we bind these classes using `pybind11`, the library ensures that the correct overridden method is called, even when accessed through a base class pointer or reference.

```cpp
Animal *get_animal() {
    return new Dog("Buddy");
}

PYBIND11_MODULE(example, m) {
    // ... previous bindings ...

    m.def("get_animal", &get_animal, pybind11::return_value_policy::take_ownership);
}
```

In the Python side, when we call `get_animal().sound()`, it will correctly return `"Woof!"`, demonstrating that `pybind11` is correctly handling the dynamic dispatch of the virtual function.

## Exception Handling

Let's consider a simple function in C++ that throws a standard exception:

```cpp
#include <pybind11/pybind11.h>
#include <stdexcept>

void throw_exception() {
    throw std::runtime_error("An error occurred!");
}

PYBIND11_MODULE(example, m) {
    m.def("throw_exception", &throw_exception);
}
```

When this function is called from Python, `pybind11` will catch the C++ exception and translate it into a Python `RuntimeError` exception with the same error message. This allows Python code to catch and handle the exception using standard Python `try`-`except` blocks.

Similarly, `pybind11` can translate Python exceptions into C++ exceptions when calling Python functions from C++.

## Multithreading

Consider a scenario where we want to run a C++ function in a separate thread and periodically update a Python object with the results.

```cpp
#include <pybind11/pybind11.h>
#include <thread>

void threaded_function(pybind11::object callback) {
    for (int i = 0; i < 5; ++i) {
        std::this_thread::sleep_for(std::chrono::seconds(1));
        // Acquire GIL before calling Python code
        pybind11::gil_scoped_acquire acquire;
        callback(i);
    }
}

PYBIND11_MODULE(example, m) {
    m.def("threaded_function", &threaded_function);
}
```

In the code above, before calling the Python `callback` function, we acquire the GIL using `pybind11::gil_scoped_acquire`. This ensures that the Python callback is executed safely, even though it's being called from a separate C++ thread. When the `gil_scoped_acquire` object goes out of scope, the GIL is automatically released, allowing other threads to execute Python code.

## In-depth Inheritance with Pybind11

Inheritance is a cornerstone of object-oriented programming, allowing for the creation of a new class that is based on an existing class. The new class inherits attributes and behaviors from the existing class. In C++, this is achieved using base and derived classes.

When exposing C++ classes to Python using `pybind11`, it's essential to ensure that the inheritance hierarchy is correctly represented in the Python bindings. This allows Python to understand and utilize the class hierarchy just as if it were defined in Python itself.

Let's delve deeper into this with a comprehensive example.

### C++ Definitions and Pybind11 Bindings

```cpp
#include <pybind11/pybind11.h>

class Vehicle {
public:
    Vehicle(const std::string &brand, double speed) : brand(brand), speed(speed) {}
    virtual std::string move() const { return "The vehicle moves."; }
protected:
    std::string brand;
    double speed;
};

class Car : public Vehicle {
public:
    Car(const std::string &brand, double speed, int wheels) : Vehicle(brand, speed), wheels(wheels) {}
    std::string move() const override { return "The car drives on roads."; }
private:
    int wheels;
};

class ElectricCar : public Car {
public:
    ElectricCar(const std::string &brand, double speed, int wheels, double battery_capacity) : Car(brand, speed, wheels), battery_capacity(battery_capacity) {}
    std::string move() const override { return "The electric car drives silently on roads."; }
private:
    double battery_capacity;
};

class Boat : public Vehicle {
public:
    Boat(const std::string &brand, double speed, int propellers) : Vehicle(brand, speed), propellers(propellers) {}
    std::string move() const override { return "The boat sails on water."; }
private:
    int propellers;
};

PYBIND11_MODULE(vehicles, m) {
    pybind11::class_<Vehicle>(m, "Vehicle")
        .def(pybind11::init<const std::string &, double>())
        .def("move", &Vehicle::move);

    pybind11::class_<Car, Vehicle>(m, "Car")
        .def(pybind11::init<const std::string &, double, int>())
        .def("move", &Car::move);

    pybind11::class_<ElectricCar, Car>(m, "ElectricCar")
        .def(pybind11::init<const std::string &, double, int, double>())
        .def("move", &ElectricCar::move);

    pybind11::class_<Boat, Vehicle>(m, "Boat")
        .def(pybind11::init<const std::string &, double, int>())
        .def("move", &Boat::move);
}
```

In the code above, we've defined our classes and their inheritance hierarchy. We then use `pybind11` to expose these classes to Python. Notice how the derived classes are specified to inherit from their respective base classes in the `pybind11::class_` template.

### Python Usage with Inherited Classes

Once the C++ classes are exposed to Python using `pybind11`, we can seamlessly use them in Python as if they were native Python classes.

```python
# Import the vehicles module (assuming it's compiled and available)
import vehicles

# Create an instance of each class
v = vehicles.Vehicle("Generic", 50)
c = vehicles.Car("Toyota", 120, 4)
e = vehicles.ElectricCar("Tesla", 150, 4, 100)
b = vehicles.Boat("Yacht", 30, 2)

# Call the move method for each instance
print(v.move())  # Output: The vehicle moves.
print(c.move())  # Output: The car drives on roads.
print(e.move())  # Output: The electric car drives silently on roads.
print(b.move())  # Output: The boat sails on water.
```

### Explanation:

1. **Vehicle Class**: This is the base class with attributes `brand` and `speed`. It has a virtual method `move()` that returns a generic movement message.

2. **Car Class**: Derived from `Vehicle`, it introduces an additional attribute `wheels` and overrides the `move()` method to provide a car-specific movement message.

3. **ElectricCar Class**: Derived from `Car`, it introduces an attribute `battery_capacity` and further overrides the `move()` method to provide a message specific to electric cars.

4. **Boat Class**: Derived from `Vehicle`, it introduces an attribute `propellers` and overrides the `move()` method to provide a boat-specific movement message.

When we create instances of these classes in Python and call their `move()` methods, the correct overridden method is called due to the dynamic dispatch mechanism, giving us the expected output.

The beauty of `pybind11` is that it allows us to work with C++ classes in Python as if they were native Python classes, respecting inheritance, polymorphism, and other OOP principles.

## In-depth Polymorphism with Pybind11

Polymorphism, derived from the Greek words 'poly' (many) and 'morph' (form), is a fundamental concept in object-oriented programming that allows objects of different classes to be treated as if they were objects of a common superclass. The most common use of polymorphism is when a parent class reference is used to refer to a child class object.

In C++, polymorphism is achieved using virtual functions, which can be overridden in derived classes. When integrating C++ with Python using `pybind11`, it's essential to ensure that polymorphic behavior is correctly represented in the Python bindings.

Let's delve deeper into this with a comprehensive example.

### C++ Definitions and Pybind11 Bindings for Polymorphism

```cpp
#include <pybind11/pybind11.h>
#include <cmath> // for M_PI

class Shape {
public:
    virtual double area() const { return 0.0; }
};

class Circle : public Shape {
public:
    Circle(double radius) : radius(radius) {}
    double area() const override { return M_PI * radius * radius; }
private:
    double radius;
};

class Rectangle : public Shape {
public:
    Rectangle(double width, double height) : width(width), height(height) {}
    double area() const override { return width * height; }
private:
    double width, height;
};

PYBIND11_MODULE(shapes, m) {
    pybind11::class_<Shape>(m, "Shape")
        .def("area", &Shape::area);

    pybind11::class_<Circle, Shape>(m, "Circle")
        .def(pybind11::init<double>())
        .def("area", &Circle::area);

    pybind11::class_<Rectangle, Shape>(m, "Rectangle")
        .def(pybind11::init<double, double>())
        .def("area", &Rectangle::area);
}
```

In the code above, we've defined our `Shape`, `Circle`, and `Rectangle` classes. The `Shape` class has a virtual method `area()`, which is overridden in both the `Circle` and `Rectangle` derived classes. We then use `pybind11` to expose these classes to Python. Notice how the derived classes are specified to inherit from the `Shape` base class in the `pybind11::class_` template.

### Python Usage with Polymorphic Classes

Once the C++ classes are exposed to Python using `pybind11`, we can utilize the power of polymorphism seamlessly in Python.

```python
# Import the shapes module (assuming it's compiled and available)
import shapes

# Create an instance of each class
circle = shapes.Circle(5)
rectangle = shapes.Rectangle(4, 6)

# Use a function to demonstrate polymorphism
def print_area(shape):
    print(f"Area of the shape: {shape.area()}")

# Call the function with different shapes
print_area(circle)  # Output: Area of the shape: 78.53981633974483
print_area(rectangle)  # Output: Area of the shape: 24.0
```

### Explanation:

1. **Shape Class**: This is the base class with a virtual method `area()` that returns the area of the shape. By default, it returns `0.0`.

2. **Circle Class**: Derived from `Shape`, it introduces an attribute `radius` and overrides the `area()` method to calculate the area of the circle using the formula \( \pi r^2 \).

3. **Rectangle Class**: Derived from `Shape`, it introduces attributes `width` and `height` and overrides the `area()` method to calculate the area of the rectangle using the formula \( width \times height \).

The function `print_area` demonstrates polymorphism. It accepts a `Shape` reference and calls the `area()` method. Depending on the actual object passed to the function, the appropriate `area()` method is called, demonstrating dynamic dispatch.

This behavior showcases the power of polymorphism, where a single function can operate on different types of objects, and the correct method is determined at runtime.

## In-depth Exception Handling with Pybind11

Exception handling is a crucial aspect of any programming language, allowing developers to manage and respond to runtime errors gracefully. In C++, exceptions are handled using the `try`, `catch`, and `throw` keywords. When integrating C++ with Python using `pybind11`, it's essential to ensure that C++ exceptions are correctly translated into Python exceptions and vice versa.

`pybind11` provides built-in support for translating C++ exceptions into Python exceptions and back. This ensures that when a C++ function throws an exception, it can be caught and handled in Python, and similarly, a Python exception can be caught and handled in C++.

Let's delve deeper into this with a comprehensive example.

### C++ Definitions and Pybind11 Bindings for Exception Handling

```cpp
#include <pybind11/pybind11.h>

double divide(double numerator, double denominator) {
    if (denominator == 0.0) {
        throw std::runtime_error("Division by zero!");
    }
    return numerator / denominator;
}

PYBIND11_MODULE(math_operations, m) {
    m.def("divide", &divide);
}
```

In the code above, we've defined a `divide` function that throws a `std::runtime_error` exception if the denominator is zero. We then use `pybind11` to expose this function to Python. When this function is called from Python with a denominator of zero, it will raise a Python `RuntimeError` exception with the message "Division by zero!".

### Python Usage with Exception Handling

Once the C++ function is exposed to Python using `pybind11`, we can handle exceptions in Python just as we would with native Python functions.

```python
# Import the math_operations module (assuming it's compiled and available)
import math_operations

# Use the divide function
try:
    result = math_operations.divide(10, 0)
    print(result)
except RuntimeError as e:
    print(f"Caught an exception: {e}")

# Output: Caught an exception: Division by zero!
```

### Explanation:

1. **divide Function**: This function takes two arguments, the numerator and the denominator. If the denominator is zero, it throws a `std::runtime_error` exception.

2. **Exception Handling in Python**: When calling the `divide` function from Python, we wrap the call in a `try` block. If the function throws an exception (in this case, due to division by zero), the exception is caught in the `except` block. The C++ exception `std::runtime_error` is automatically translated to a Python `RuntimeError` by `pybind11`.

3. **Output**: The Python code catches the exception and prints the error message "Division by zero!".

This example showcases how `pybind11` seamlessly translates C++ exceptions into Python exceptions, allowing for consistent and intuitive error handling across both languages.

## In-depth Multithreading with Pybind11

Multithreading is a powerful technique that allows a program to run multiple threads concurrently, potentially improving performance and responsiveness. In C++, the `<thread>` header provides functionalities to manage threads. When integrating C++ with Python using `pybind11`, it's essential to be aware of the Global Interpreter Lock (GIL) in Python, which prevents multiple native threads from executing Python bytecodes at once.

However, `pybind11` provides utilities to work around the GIL, allowing C++ threads to run concurrently while ensuring that only one thread interacts with the Python interpreter at a time.

Let's delve deeper into this with a comprehensive example.

### C++ Definitions and Pybind11 Bindings for Multithreading

```cpp
#include <pybind11/pybind11.h>
#include <pybind11/stl.h>
#include <thread>
#include <vector>

double compute_sum(const std::vector<double>& numbers) {
    double sum = 0.0;
    for (double num : numbers) {
        sum += num;
    }
    return sum;
}

double parallel_sum(const std::vector<double>& numbers, int num_threads) {
    int chunk_size = numbers.size() / num_threads;
    std::vector<std::thread> threads;
    std::vector<double> results(num_threads);

    for (int i = 0; i < num_threads; ++i) {
        threads.emplace_back([&, i]() {
            results[i] = compute_sum(std::vector<double>(numbers.begin() + i * chunk_size, numbers.begin() + (i + 1) * chunk_size));
        });
    }

    for (auto& t : threads) {
        t.join();
    }

    return compute_sum(results);
}

PYBIND11_MODULE(threading_example, m) {
    m.def("parallel_sum", &parallel_sum, "Compute the sum of a list of numbers using multiple threads.");
}
```

In the code above, we've defined a `compute_sum` function that calculates the sum of a list of numbers. We also have a `parallel_sum` function that splits the list into smaller chunks and processes each chunk in a separate thread. We then use `pybind11` to expose the `parallel_sum` function to Python. This function will allow us to leverage the power of multithreading from Python, even though the actual computation is done in C++.

### Python Usage with Multithreading

Once the C++ function is exposed to Python using `pybind11`, we can leverage the power of multithreading directly from Python.

```python
# Import the threading_example module (assuming it's compiled and available)
import threading_example

# Generate a large list of numbers
numbers = [i * 0.5 for i in range(1, 100001)]

# Use the parallel_sum function with 4 threads
result = threading_example.parallel_sum(numbers, 4)
print(f"Sum of numbers: {result}")

# Expected Output: Sum of numbers: 2500025000.0
```

### Explanation:

1. **compute_sum Function**: This function calculates the sum of a list of numbers sequentially.

2. **parallel_sum Function**: This function splits the list of numbers into smaller chunks based on the number of threads specified. Each chunk is then processed in a separate thread using the `compute_sum` function. Once all threads have completed their computation, the results are aggregated to produce the final sum.

3. **Python Usage**: In the Python code, we generate a large list of numbers and then use the `parallel_sum` function to compute their sum using 4 threads. The result is then printed.

This example showcases how `pybind11` allows us to leverage the power of multithreading in C++ from Python. The actual computation is done in C++, but the Python code can control the number of threads and access the result seamlessly.

## Combining Multithreading and Exception Handling with Pybind11

When combining multithreading with exception handling, it's crucial to ensure that exceptions thrown in a thread are captured and handled appropriately. This can be a bit tricky since exceptions thrown in one thread cannot be caught in another. However, with some careful design, we can propagate exceptions from worker threads back to the main thread.

Let's consider an example where we compute the inverse of a list of numbers in parallel. If any number in the list is zero, computing its inverse would result in a division by zero error. We want to capture this exception and handle it gracefully.

We will:

1. Define the C++ function that computes the inverse of numbers in parallel and handles exceptions.
2. Expose it to Python using `pybind11`.
3. Demonstrate its usage in Python.
4. Provide a detailed explanation.

Let's start by defining and exposing the function.

## Combining Multithreading with Exception Handling

When working with multithreading, it's crucial to handle exceptions correctly to ensure that errors in one thread don't crash the entire application or leave it in an inconsistent state. Combining multithreading with exception handling can be challenging, but it's essential for robust and resilient applications.

Let's consider an example where we compute the reciprocal of a list of numbers using multiple threads. If any number in the list is zero, computing its reciprocal will result in a division by zero error. We'll handle this exception and ensure that the program continues to process the remaining numbers.

### Example:

We have a function `parallel_reciprocal` that takes a list of numbers and returns a list containing the reciprocal of each number. The function splits the list into smaller chunks and processes each chunk in a separate thread. If a division by zero error occurs in any thread, it's caught, and the result for that number is set to `inf` (infinity).

Let's start by defining and exposing the function.

### C++ Definitions and Pybind11 Bindings for Multithreading with Exception Handling

```cpp
#include <pybind11/pybind11.h>
#include <pybind11/stl.h>
#include <thread>
#include <vector>
#include <stdexcept>

// Function to compute the reciprocal of a number
double reciprocal(double number) {
    if (number == 0.0) {
        throw std::runtime_error("Division by zero!");
    }
    return 1.0 / number;
}

// Function to compute the reciprocal of a list of numbers using multiple threads
std::vector<double> parallel_reciprocal(const std::vector<double>& numbers, int num_threads) {
    int chunk_size = numbers.size() / num_threads;
    std::vector<std::thread> threads;
    std::vector<double> results(numbers.size());

    for (int i = 0; i < num_threads; ++i) {
        threads.emplace_back([&, i]() {
            for (int j = i * chunk_size; j < (i + 1) * chunk_size && j < numbers.size(); ++j) {
                try {
                    results[j] = reciprocal(numbers[j]);
                } catch (const std::runtime_error&) {
                    results[j] = std::numeric_limits<double>::infinity();
                }
            }
        });
    }

    for (auto& t : threads) {
        t.join();
    }

    return results;
}

PYBIND11_MODULE(threading_exceptions_example, m) {
    m.def("parallel_reciprocal", &parallel_reciprocal, "Compute the reciprocal of a list of numbers using multiple threads and handle exceptions.");
}
```

In the code above:

- The `reciprocal` function computes the reciprocal of a number. If the number is zero, it throws a `std::runtime_error` exception.
- The `parallel_reciprocal` function splits the list of numbers into smaller chunks and processes each chunk in a separate thread. If a division by zero error occurs in any thread, the exception is caught, and the result for that number is set to `inf` (infinity).
- We then use `pybind11` to expose the `parallel_reciprocal` function to Python.

### Python Usage with Multithreading and Exception Handling

Once the C++ function is exposed to Python using `pybind11`, we can use it to compute the reciprocals of numbers, even if the list contains zero, and handle exceptions gracefully.

```python
# Import the threading_exceptions_example module (assuming it's compiled and available)
import threading_exceptions_example

# List of numbers including zero
numbers = [1.0, 2.0, 0.0, 4.0, 5.0]

# Use the parallel_reciprocal function with 3 threads
results = threading_exceptions_example.parallel_reciprocal(numbers, 3)
print(f"Reciprocals: {results}")

# Expected Output: Reciprocals: [1.0, 0.5, inf, 0.25, 0.2]
```

### Explanation:

1. **reciprocal Function**: This function computes the reciprocal of a number. If the number is zero, it throws a `std::runtime_error` exception indicating a division by zero error.

2. **parallel_reciprocal Function**: This function splits the list of numbers into smaller chunks and processes each chunk in a separate thread. If a division by zero error occurs in any thread (i.e., when computing the reciprocal of zero), the exception is caught, and the result for that number is set to `inf` (infinity).

3. **Python Usage**: In the Python code, we have a list of numbers that includes zero. We use the `parallel_reciprocal` function to compute the reciprocals of these numbers using 3 threads. The result is then printed. As expected, the reciprocal of zero is represented as `inf`.

This example demonstrates how `pybind11` allows us to combine multithreading with exception handling. The actual computation and exception handling are done in C++, but the Python code can control the number of threads and access the results seamlessly.